In [1]:
import os
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
from langchain_teddynote.tools.tavily import TavilySearch
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from typing import Annotated, TypedDict
import ast
import re

In [18]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import os

# Step 1: Load CSV data
file_path = "data/media.csv"  # This should be the uploaded CSV file
df = pd.read_csv(file_path, header=None)

# Step 2: Define column names manually since no header in the file
df.columns = [
    "media_id", "media_name", "location", "size", "duration", "media_type",
    "operating_hours", "is_digital", "slot_count", "is_available", "unit_price",
    "location_description", "image_day", "image_night", "image_map",
    "population_target", "media_characteristics", "case_examples"
]

# Step 3: Prepare embedding model using HuggingFace (MiniLM)
class BERTSentenceEmbedding:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def embed_documents(self, texts):
        return [self._embed(text) for text in texts]

    def embed_query(self, text):
        return self._embed(text)

    def _embed(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze(0)
        return cls_embedding.cpu().numpy()

embedding_function = BERTSentenceEmbedding()

# Step 4: Construct documents for Chroma
def build_text(row):
    return f"""
    위치 설명: {row['location_description']}
    타겟: {row['population_target']}
    매체 특징: {row['media_characteristics']}
    집행 사례: {row['case_examples']}
    """

docs = []
for i, row in df.iterrows():
    doc = Document(
        page_content=build_text(row),
        metadata={
            "media_id": str(row["media_id"]),
            "media_name": row["media_name"],
            "location": row["location"],
            "media_type": row["media_type"],
            "population_target": row["population_target"],
            "media_characteristics": row["media_characteristics"],
            "case_examples": row["case_examples"]
        }
    )
    docs.append(doc)

# Step 5: Store in Chroma
chroma_collection = Chroma.from_documents(
    documents=docs,
    embedding=embedding_function,
    collection_name="media",
    persist_directory="./chroma_media"
)


In [19]:
chroma_collection.persist()

/var/folders/x8/x9mhzs_j0yx8btgk3t9m0vvc0000gn/T/ipykernel_2183/570867232.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_collection.persist()


In [20]:
print("저장된 문서 수:", chroma_collection._collection.count())  # 또는 chroma_collection._collection.count()


저장된 문서 수: 51


In [ ]:
# 에이전트: 브랜드 정보를 기반으로 크로마 DB에서 관련 매체 추천

from typing import TypedDict
from langchain_openai import ChatOpenAI

# Step 1: 입력/출력 구조 정의
class MatchingInput(TypedDict):
    brand_name: str
    recent_issue: str
    brand_description: str

class MatchingOutput(TypedDict):
    brand: dict
    recommended_package: dict
    sales_talking_points: list[str]

# Step 2: 임베딩 및 Chroma 연결
embedding_function = BERTSentenceEmbedding()
chroma_collection = Chroma(
    collection_name="media",
    embedding_function=embedding_function,
    persist_directory="./chroma_media"
)

# Step 3: LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# Step 4: 에이전트 정의
def media_matcher_agent(state: MatchingInput) -> MatchingOutput:
    # 1. 쿼리 텍스트 구성
    query_text = f"{state['recent_issue']} / {state['brand_description']}"

    # 2. 유사도 기반 검색
    results = chroma_collection.similarity_search_with_score(query_text, k=5)

    # 3. 결과 정리 (상위 3개)
    top_matches = []
    for doc, score in results[:3]:
        meta = doc.metadata
        top_matches.append({
            "media_id": meta["media_id"],
            "media_name": meta["media_name"],
            "location": meta["location"],
            "media_type": meta["media_type"],
            "match_score": int((1 - score) * 100),
            "individual_reason": f"{state['recent_issue'].split()[2]} 지역과 유사한 위치 + 타겟 적합성"
        })

    # 4. 매칭 사유 요약
    matching_reason = f"{state['brand_name']}의 최근 마케팅 이슈와 지역/타겟 오디언스를 기준으로 선정된 매체 조합입니다."

    # 5. 세일즈 포인트 생성
    prompt = f"""
    브랜드: {state['brand_name']}
    이슈: {state['recent_issue']}
    매체: {', '.join([m['media_name'] for m in top_matches])}

    위 정보를 바탕으로 광고주에게 제안할 세일즈 포인트 3가지를 만들어줘.
    """
    sales_point = llm.invoke(prompt).content.strip().split("\n")

    # 6. 최종 출력 구조 반환
    return {
        "brand": {
            "name": state["brand_name"],
            "recent_issue": state["recent_issue"],
            "target_audience": "타겟 오디언스 자동 추출 예정"
        },
        "recommended_package": {
            "package_name": f"{state['recent_issue'].split()[2]} 프리미엄 패키지",
            "media_list": top_matches,
            "matching_reason": matching_reason
        },
        "sales_talking_points": [line.strip("- ").strip() for line in sales_point if line.strip()]
    }


/var/folders/x8/x9mhzs_j0yx8btgk3t9m0vvc0000gn/T/ipykernel_2183/3638289297.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_collection = Chroma(


In [25]:
response = media_matcher_agent({
    "brand_name": "더바넷",
    "recent_issue": "2025년 3월 9일: 서울 잠실 롯데월드몰에 국내 첫 팝업스토어 오픈",
    "brand_description": "2021년 론칭한 캐주얼 브랜드로, 20·30세대 고객에게 가장 트렌디한 브랜드로 손꼽히며, 가방과 모자, 액세서리를 포함한 다양한 상품을 선보인다."
})
print(response)


{'brand': {'name': '더바넷', 'recent_issue': '2025년 3월 9일: 서울 잠실 롯데월드몰에 국내 첫 팝업스토어 오픈', 'target_audience': '타겟 오디언스 자동 추출 예정'}, 'recommended_package': {'package_name': '9일: 프리미엄 패키지', 'media_list': [{'media_id': '17', 'media_name': '가로변 버스쉘터 강남대로', 'location': '서울시 강남구 강남대로 일대', 'media_type': '버스정류장 쉘터 광고', 'match_score': -386, 'individual_reason': '9일: 지역과 유사한 위치 + 타겟 적합성'}, {'media_id': '12', 'media_name': '홍대입구역 스칼렛 전광판', 'location': '서울시 마포구 양화로 148', 'media_type': '벽면형 세로 사이니지', 'match_score': -390, 'individual_reason': '9일: 지역과 유사한 위치 + 타겟 적합성'}, {'media_id': '2', 'media_name': '서울 고속버스터미널 (경부선)', 'location': '서울시 서초구 신반포로 194', 'media_type': '옥상형 가로 사이니지', 'match_score': -392, 'individual_reason': '9일: 지역과 유사한 위치 + 타겟 적합성'}], 'matching_reason': '더바넷의 최근 마케팅 이슈와 지역/타겟 오디언스를 기준으로 선정된 매체 조합입니다.'}, 'sales_talking_points': ['다음은 더바넷의 팝업스토어 오픈을 홍보하기 위한 세일즈 포인트 3가지입니다:', '1. **최초의 팝업스토어 경험 제공**:', '더바넷은 2025년 3월 9일 서울 잠실 롯데월드몰에 국내 최초의 팝업스토어를 오픈합니다. 소비자들에게 특별한 경험과 함께 한정판 제품을 체험할 수 있는 기회를 제